<a href="https://colab.research.google.com/github/OlenaBugaiova/advisor-to-farmers/blob/main/code/example/rag_for_question_answering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task Description

<img src=attachment:8f331cdf-28f2-4161-976a-d9fe6a4520bb.png width = "800">

This notebook provides an example of how ChatGPT can be augumented with related context as AI based advisor in agriculture in norwegian.

# Import Libraries

In [ ]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np
import pandas as pd
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', None)

import os
import json

from openai import OpenAI
import ast
import pdb

In [ ]:
RANDOM_STATE = 311

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret('OPENAI_API_KEY')

In [ ]:
client = OpenAI(
    api_key = api_key
)

# Load the data

In [ ]:
directory_name = '../input/about-agriculture-in-norwegian-language/'
combined_file_name = 'text_on_agriculture.json'

In [ ]:
with open(directory_name + combined_file_name, 'r') as file_name:
    text_combined = json.load(file_name)

In [ ]:
type(text_combined)

dict

# Simple promt

In [ ]:
# question = 'Er ferske, rå grønnsaker sunnere og mer næringsrike enn frosne grønnsaker?'
question = 'Bør bøndene vurdere bioenergi?'
question

'Bør bøndene vurdere bioenergi?'

In [ ]:
system_message = 'Du er en hjelpsom assistent som gir råd innen landbruk og svarer på spørsmål fra bønder på norsk'
system_message

'Du er en hjelpsom assistent som gir råd innen landbruk og svarer på spørsmål fra bønder på norsk'

In [ ]:
response = client.chat.completions.create(
    model = 'gpt-4o-mini',
    messages = [
        {
            'role': 'system',
            'content': system_message
        },
        {'role': 'user', 'content': question}
    ]
)
response

ChatCompletion(id='chatcmpl-AmmeAREfp7KM6ORiY9hSXDtV6Jro1', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Ja, bønder bør absolutt vurdere bioenergi som et alternativ. Her er noen grunner til hvorfor det kan være fordelaktig:\n\n1. **Bærekraftig energi**: Bioenergi er en fornybar energikilde som kan bidra til å redusere avhengigheten av fossile brensler. Dette kan være en viktig del av en bærekraftig landbrukspraksis.\n\n2. **Avfallshåndtering**: Mange landbrukssubsidiar kan produsere biomasseavfall, for eksempel fra avlinger eller husdyrgjødsel. Disse materialene kan brukes til å produsere bioenergi, som biogass eller biodiesel, noe som kan redusere avfall og kostnader knyttet til avfallshåndtering.\n\n3. **Ekstra inntektskilde**: Ved å produsere bioenergi kan bønder diversifisere inntektene sine. De kan selge energi tilbake til nettet eller bruke den til eget forbruk, noe som kan redusere driftskostnader.\n\n4. **Klima- og miljøfo

In [ ]:
response.choices[0].message.content

'Ja, bønder bør absolutt vurdere bioenergi som et alternativ. Her er noen grunner til hvorfor det kan være fordelaktig:\n\n1. **Bærekraftig energi**: Bioenergi er en fornybar energikilde som kan bidra til å redusere avhengigheten av fossile brensler. Dette kan være en viktig del av en bærekraftig landbrukspraksis.\n\n2. **Avfallshåndtering**: Mange landbrukssubsidiar kan produsere biomasseavfall, for eksempel fra avlinger eller husdyrgjødsel. Disse materialene kan brukes til å produsere bioenergi, som biogass eller biodiesel, noe som kan redusere avfall og kostnader knyttet til avfallshåndtering.\n\n3. **Ekstra inntektskilde**: Ved å produsere bioenergi kan bønder diversifisere inntektene sine. De kan selge energi tilbake til nettet eller bruke den til eget forbruk, noe som kan redusere driftskostnader.\n\n4. **Klima- og miljøfordeler**: Bruk av bioenergi kan bidra til å redusere klimagassutslippene fra landbruket. Dette kan være en viktig faktor i møte med klimaendringer og i arbeidet

# Text Embedding

In [ ]:
text_combined_df = pd.DataFrame(text_combined.items(), columns = ['title', 'text'])
text_combined_df.head(2)

,title,text
0,Fossilfri maskinpark – muligheter for landbruket og status,"\n Et av satsingsområdene i landbrukets klimaplan er utfasing av fossilt drivstoff og overgang til biodrivstoff eller maskiner som går på elektrisitet, biogass eller hydrogen. Mål om klimagassreduksjon er på 600 000 – 1 470 000t CO₂ ekvivalenter.\n «Klimakur 2030» forteller om ca. 160 000 registrerte traktorer i jordbruket som står for ca. 90 % av de totale utslippene fra næringens maskinpark. Et grovt anslag på CO₂-utslippet fra fossilt drivstoff brukt i traktorene kan beregnes til ca. 550 000t CO₂ ekv.\n Bruk av biodrivstoff ansees som et smertefritt tiltak, siden den ikke gir store driftsmessige eller tekniske konsekvenser. Avanserte biodrivstoff er mest brukt i Norge, og lages i all hovedsak av slakteavfall og brukt frityrolje. Konvensjonelle biodrivstoff baserer seg på landbruksvekster som raps og har fått en langt mindre avsetning i de senere årene. Det skal nevnes at prislappen på biodrivstoff er ca. 30 % høyere pr. liter enn fossil diesel. Pr. dags dato er det heller ikke noen norsk produksjon av biodiesel og tilgjengelig drivstoff blir produsert utenlandsk og underligger en streng sertifisering. \n Elektrisitet kan være et alternativ for mindre maskiner, som minilaster eller teleskoplastere. For traktorer vil dagens kapasitet på batteripakka være for liten for å gjennomføre de tyngre oppgavene i gårdsdrifta. Selv om en produsent tilbyr en 100Hk batteridreven smalsportraktor vil ikke denne traktoren være et alternativ for jordbrukeren og trolig passe mer til lettere oppgaver i kommunal drift. \n Gassdrevne traktorer med CBG (komprimert biogass)\n produseres i serie, et fåtall finnes i Norge pr. dags dato. New Holland T6 modell utstyrte med en 160Hk motor og etter opplysninger fra produsenten skal det komme en sterkere modell med 270Hk i slutten av 2024. CBG tankene på disse traktorene kan lagre maks 47kg CBG som tilsvarer ca. 61l diesel på energibasis. Prisen på CBG traktorer er ca. 25 – 30 % høyere enn hos tilsvarende dieselmodeller. Lønnsomheten for CBG som drivstoff forutsetter hyppig og jevn avsetning av gassen.\n Hydrogen som drivstoff kan være et fremtidig alternativ. Imidlertid kreves det en kostbar prosess for å lage hydrogen gjennom elektrolysen. Andre faktorer som lite energitetthet pr. volum i både gass – og flytende form og et lite utbygd distribusjonsnett tilsier at kommersiell bruk av den typen drivstoff neppe vil være et alternativ frem til 2030.\n Økte investerings- og driftskostnader ved overgang til fossilfri drivstoff må følges med tilskuddsordninger eller kostnadskompensasjoner. Vi registrerer et levende CO₂-marked for drivstoffdistributører i flere land i Europa, som gjør det lønnsomt å erstatte fossil med fossilfri drivstoff. Markedsprisen på CO₂ kvoten (€/t CO₂ – ekv) graderes etter substrat for å lage fossilfri drivstoff, der f.eks. husdyrgjødsel har høyest gradering og dermed kan gi høyere inntekter enn plantebasert drivstoff. Andre tiltak i andre EU-land er skattelettelse på fossilfri drivstoff.\n Det er vel å merke at landbruket kan produsere egen CBG fra egen husdyrgjødsel som da kan brukes i egne traktorer. Utenlandske erfaringer viser til at teknikken er moden både på oppgraderingsanlegg for mindre biogassanlegg, traktorer og private fyllestasjoner."
1,Strømproduksjon fra husdyrgjødsel – enkel betraktning,"\n Husdyrgjødsel ansees som bøndenes gull fordi den har en bestemt gjødselverdi og sørger for at organisk materiale tilføres jorda som mat for mikroorganismene oppå og under torva. \n Den siste tiden har det vært mange spørsmål om husdyrgjødsel, biogass og strømproduksjon. Nedenfor står noen enkle betraktninger om denne tematikken som konkluderes med at husdyrgjødsel fungerer som strømkilde.\n I husdyrgjødsel ligger det en god del ufordøyde plantefibre og kornrester som har et gasspotensiale. Gassutbytte fra husdyrgjødsel er avhengig av hva produksjonsdyret fôres med. Eksempelvis vil gjødsla fra ei høytytende melke

In [ ]:
def get_embedding(text, model = 'text-embedding-3-small'):
    text = text.replace('/n', '. ')
    embedding = client.embeddings.create(input = [text], model = model).data[0].embedding
    return embedding

In [ ]:
get_embedding(text_combined_df['text'].iloc[0])[0:10]

[-0.0008387480047531426,
 0.02726868912577629,
 0.024653296917676926,
 0.013194872997701168,
 0.00523078627884388,
 0.03185634687542915,
 -0.02148052491247654,
 0.02960539236664772,
 0.0736168846487999,
 0.023688601329922676]

For simplicity of this example, we take only the first 50 articles

In [ ]:
text_combined_embeddings_df = text_combined_df.head(50).copy()

In [ ]:
%%time
text_combined_embeddings_df['embeddings'] = text_combined_embeddings_df['text'].apply(get_embedding)

CPU times: user 193 ms, sys: 15.8 ms, total: 208 ms
Wall time: 13.9 s


In [ ]:
text_combined_embeddings_df.to_csv('./website_text_with_embeddings.csv', index = False)

In [ ]:
question_embedding = get_embedding(question)
question, question_embedding[0:10]

('Bør bøndene vurdere bioenergi?',
 [-0.014635080471634865,
  0.02086324244737625,
  0.005180048290640116,
  0.010354594327509403,
  -0.005931058898568153,
  0.07390165328979492,
  -0.043046943843364716,
  -0.012588370591402054,
  0.07236111909151077,
  -0.02636515349149704])

# Similar topics

In [ ]:
def find_distance(page_embedding, question_embedding = question_embedding):
    distance = np.dot(page_embedding, question_embedding)
    return distance

In [ ]:
text_combined_embeddings_df['distance'] = text_combined_embeddings_df['embeddings'].apply(find_distance)

In [ ]:
text_combined_embeddings_df.sort_values('distance', ascending = False, inplace = True)

In [ ]:
text_combined_embeddings_df[['title', 'text', 'distance']].head()

title  \
3                         Bioenergi er lønnsomt   
4                   Garden som energiprodusent?   
8  Biogassproduksjon på gården - den nye oljen?   
2     Høgaktuelt med energiproduksjon på garden   
7       Status solceller: Er det fortsatt lurt?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

Create context

In [ ]:
context = text_combined_embeddings_df['text'].iloc[0] + '\n' + text_combined_embeddings_df['text'].iloc[1] + '\n' + text_combined_embeddings_df['text'].iloc[2] + '\n' + text_combined_embeddings_df['text'].iloc[3] + '\n'
context

'\n  Det har vært skrevet metervis av artikler og fagskrifter om det grønne skifte. Alt er grønt, og et av de viktigste tiltakene for Norge skal komme fra den grønne skogen. Det er sant, men det skjer også i dag, halvparten av CO2 utslippet på fastlands Norge blir tatt opp av skogen. Videre mål blir å redusere CO2 utslippet. Den enkleste og mest lønnsomme måten man gjør det på er å velge bioenergi som oppvarmingskilde. Nå har alle gårdeiere i Sør-Norge mulighet til å få gratis befaring og hjelp til å se på mulighetene på egen gård.\n  Eiere av landbrukseiendommer har et kjempefortrinn. De kan være selvforsynt med energi. En gård kan hente varmen fra bioenergi hentet i egen skog, og låvetaket er velegnet for solcellepaneler. Og med dette spare energikostnader. Du blir ikke lenger påvirket av høye nettleier eller høye strømpriser på gitte tider av døgnet. I snitt betaler du kanskje bare 50 øre/kWh for biovarme. Merkostnaden ved å bruke litt mer flis altså øke varmebehovet er ikke mer enn

# Chat response after receiving more context

In [ ]:
system_message = system_message + ' Du vet følgende: ' + context

In [ ]:
question

'Bør bøndene vurdere bioenergi?'

In [ ]:
response = client.chat.completions.create(
    model = 'gpt-4o-mini',
    messages = [
        {
            'role': 'system',
            'content': system_message
        },
        {'role': 'user', 'content': question}
    ]
)

In [ ]:
response.choices[0].message.content

'Ja, bønder bør absolutt vurdere bioenergi som et alternativ for å produsere egen energi. Det er flere gode grunner til dette:\n\n1. **Kostnadsbesparelser**: Ved å bruke bioenergi kan bønder redusere energikostnadene betydelig, spesielt i tider med høye strømpriser. Biovarme kan være mye rimeligere sammenlignet med vanlige energikilder.\n\n2. **Selvforsyning**: Bioenergi gir bønder muligheten til å bli mer selvforsynte når det gjelder energi. Dette er spesielt nyttig i landbruket, hvor man kan utnytte ressurser som trevirke fra egen skog eller husdyrgjødsel til energiproduksjon.\n\n3. **Miljøfordeler**: Bruken av bioenergi er en CO2-nøytral løsning, da plantene som brukes til biomasse tar opp CO2 under veksten. Dette bidrar til å redusere klimagassutslippene.\n\n4. **Energiproduksjon og -salg**: Bønder kan ikke bare bruke bioenergi til eget forbruk, men også produsere energi for salg. Dette kan skape en ny inntektskilde for gården.\n\n5. **Støtteordninger**: Det er flere støtteordninge